# Project 3 - Article Parsing Toolset

## Team 3 Members

- Matthew Dunbar
- Jeffrei Cher
- Basil James

## Project Problem

To parse article content and extract various data from the article contents

- _**Article Classification into a set of pre-defined categories**_
  (a prediction model)

For Sports Articles:
- _**to identify the sport, and extract game/match scores if present**_
- _(to provide stats summaries if present?)_

## Learning Goal

Develop experience:

- Building and deploying models to leverage in real world applications
- Leveraging custom training of LLMs to provide article analysis.
- Using a tool-based (extensible toolbox) approach to provide multiple analytics features


## Dataset

- https://www.kaggle.com/datasets/fabiochiusano/medium-articles  
[size: 190k+, categories: multiple tags per article, large set, Includes titles, full articles, and URLs]

### Retrieve dataset

In [1]:
! kaggle datasets download -d fabiochiusano/medium-articles -p ./data --unzip

Dataset URL: https://www.kaggle.com/datasets/fabiochiusano/medium-articles
License(s): CC0-1.0
100%|█████████████████████████████████████████| 369M/369M [00:03<00:00, 144MB/s]
100%|█████████████████████████████████████████| 369M/369M [00:03<00:00, 119MB/s]


### Local file

In [7]:
!ls -l ./data/

total 1017916
-rw-r--r-- 1 jupyter jupyter 1042340506 Apr  2 17:18 medium_articles.csv


### Fulfill basic Dataframe dependencies

In [72]:
import os

import pandas as pd
from google.cloud import bigquery

import subprocess
import warnings

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
warnings.filterwarnings("ignore")

import json
from pprint import pprint
import markdown
# import tensorflow as tf

print(tf.version.VERSION)

2.12.0


### Load dataframe

In [117]:
# Load dataset (change filename accordingly)
df = pd.read_csv('./data/medium_articles.csv')

# Display first few rows
df.head()

title                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

### Parse DataFrame to remove unwanted columns

In [66]:
# parse the CSV data
CSV_COLUMNS = [
    "title",
    "text",
    "url",
    "authors",
    "timestamp",
    "tags",
]
LABEL_COLUMN = "text"
DEFAULTS = [["na"], ["na"], ["na"], ["na"], ["na"], ["na"]]
UNWANTED_COLS = ["title", "url", "authors", "timestamp"]

DESIRED_COLUMNS = [col for col in CSV_COLUMNS if col not in UNWANTED_COLS]
df = df[DESIRED_COLUMNS]

# Show full column width and prevent truncation
pd.set_option("display.max_colwidth", None)  # Show full text
pd.set_option("display.expand_frame_repr", False)  # Prevent wrapping

df.head()

text                                                                       tags
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           Photo by Josh Riemer on Unsplash\n\nMerry Christmas and Happy Holidays, everyone!\n\nWe just wanted everyone to know how much we appreciate everyone and how thankful we are for all our readers and writers here. We wouldn’t be anywhere without you, so thank you all for bringing informative, vulnerable, and important pieces that destigmatize mental illness and mental health.\n\nWithout further ado, here are ten of our top stories from last week, all of which were curated:\n\n“Just as the capacity to love and inspire is universal so is the capacity to hate and discourage. Irrespective of gender, race, age or religion none of us are exempt from aggressive proclivities. Those who are narcissistically disordered, and accordingly repress deep seated feelings of inferiority with inflated delusions of grandeur and superiority, are more prone to aggression and violence. They infiltrate our interactions in myriad environments from home, work, school and the cyber world. Hence, bullying does not happen in isolation. Although there is a ringleader she looks to her minions to either sanction her cruelty or look the other way.”\n\n“Even though the circumstances that brought me here were sad and challenging, I’m grateful for how this program has changed my life for the better. I can’t help but imagine what life would be like if everyone learned to accept their powerlessness over other people, prioritize their serenity, and take life one step at a time. We’ll never know, but I’d bet the world would be much happier.”\n\n“The prospect of spending a horrible Christmas, locked in on a psychiatric unit, was one of 

### Check formatting within text column

In [69]:
print(df["text"].iloc[4])  # Display only the 'text' column

You’ve heard of him, haven’t you? Phineas Gage. The railroad worker who survived an explosion that involved an iron rod piercing through his left cheek and out of his brain and skull.

Yeah.

I know.

You’re probably wondering “yeah, alright sweet. What about him?” Well, let’s just say that he was a really popular patient for the field of neuroscience (Cherry, par. 1). And what I found the most interesting about this tragic event was the science of his behavior afterward.

For those of you who don’t know much about Phineas Gage, let me fill you in with the help of my research.

Phineas Gage, 25 years old, was a railroad worker in Vermont. One day, at work, he was using an iron rod to handle explosive gun powder. As he was using the iron rod to handle the gun powder, an explosion suddenly occurred. The iron rod then went through his left cheek and brain. Fortunately, he survived and was able to talk and walk after the accident (Cherry, par. 2–3).

Why did people say that Phineas Gage wa

### Parse Data Sets from DataFrame

In [75]:
from sklearn.model_selection import train_test_split

# Split into training and temp (validation + test) set (70% train, 30% temp)
train, temp = train_test_split(df, test_size=0.3, random_state=42)

# Split the temp set into validation and test set (50% validation, 50% test of the 30%)
validate, test = train_test_split(temp, test_size=0.5, random_state=42)

# Print counts of rows in each set
print(f"Training Set Size: {train.shape[0]}")
print(f"Validation Set Size: {validate.shape[0]}")
print(f"Test Set Size: {test.shape[0]}")

Training Set Size: 134657
Validation Set Size: 28855
Test Set Size: 28856


## Solution

### Approach

LLM-based tool(s) with tool-specific training, with tool-specific engineered prompt(s).
Modular. Function-based.  Extensible.


#### Input: content of the article to analyse

(ideally, the contents directly.  alternate consideration might be to provide a URL, but that would require additional python supporting fucntions to pull, then clean up the contents prior to submission)

#### Output: variable per tool/function

### Instantiate an LLM, and set it up to support one or more functions

In [78]:
# Instantiate an LLM
from typing import Any, Callable, Optional, Tuple, Union

from google import genai
from google.cloud import bigquery
from google.genai.types import (
    FunctionDeclaration,
    GenerateContentConfig,
    GenerateContentResponse,
    Part,
    Schema,
    Tool,
)
from IPython.display import Markdown

REGION = "us-central1"
PROJECT = !(gcloud config get-value core/project)
PROJECT = PROJECT[0]

MODEL = "gemini-2.0-flash-001"

client = genai.Client(vertexai=True, location="us-central1")

# Define the ChatAgent parent class
class ChatAgent:
    def __init__(
        self,
        tools: list[Tool],
        tool_handler_fn: Callable[[str, dict], Any],
        max_iterative_calls: int = 5,
    ):
        self.tools = tools
        self.tool_handler_fn = tool_handler_fn
        self.chat_session = chat = client.chats.create(
            model=MODEL,
            config=GenerateContentConfig(tools=tools),
        )
        self.max_iterative_calls = 5

    def send_message(self, message: str) -> GenerateContentResponse:
        response = self.chat_session.send_message(message)
        # This is None if a function call was not triggered
        fn_calls = response.function_calls

        num_calls = 0
        # Reasoning loop. If fn_calls is empty then we never enter this
        # and simply return the response
        while fn_calls:
            if num_calls > self.max_iterative_calls:
                break

            # Handle the function calls
            fn_call_responses = []
            for fn_call in fn_calls:
                response = self.tool_handler_fn(
                    fn_call.name, dict(fn_call.args)
                )
                fn_call_responses.append(
                    Part.from_function_response(
                        name=fn_call.name,
                        response={
                            "content": response,
                        },
                    ),
                )
                num_calls += 1

            # Send the function call result back to the model
            response = self.chat_session.send_message(fn_call_responses)

            # If the response is another function call then we want to
            # stay in the reasoning loop and keep calling functions.
            fn_calls = response.function_calls

        return response

### Tools

#### Article Topic Classifier

input: article body  
output: one of a curated set of topic categories, based upon highest probability match

##### Create Custom Query and Response Templates

###### LLM Input

In [177]:
# Define the function declaration for the topic classifier
from pydantic import BaseModel, Extra, Field
from typing import Dict, Any
from enum import Enum

# Define an Enum for topics
class TopicEnum(str, Enum):
    arts = "arts"
    business = "business"
    entertainment = "entertainment"
    culture = "culture"
    literature = "literature"
    medicine = "medicine"
    music = "music"
    personal_development = "personal development"
    philosophy = "philosophy"
    politics = "politics"
    religion = "religion"
    science = "science"
    sports = "sports"
    technology = "technology"
    us_news = "us news"
    world = "world"

    # Use a @classmethod to safely store descriptions
    @classmethod
    def _get_descriptions(cls):
        return {
            "arts": "Cultural and creative activities, including fine arts, theater, and music.",
            "business": "The activities related to the production, distribution, and sale of goods and services.",
            "entertainment": "Media, performance, and activities designed to entertain an audience.",
            "culture": "The shared customs, arts, and social institutions of a particular group of people.",
            "literature": "Written works, books, essays, poems, et cetera, especially those considered of superior or lasting artistic merit.",
            "medicine": "The field of health and healing, including clinical practices and healthcare.",
            "music": "An art form that uses sound and rhythm to express emotions, ideas, and cultural identity.",
            "personal_development": "Activities and practices that improve awareness and identity, develop talents, and enhance the quality of life.",
            "philosophy": "The study of fundamental questions regarding existence, knowledge, ethics, reason, and the mind.",
            "politics": "The activities associated with governance, policy, and political ideologies.",
            "religion": "The system of beliefs, practices, and worship regarding a deity or deities.",
            "science": "Systematic enterprise that builds and organizes knowledge through testable explanations and predictions.",
            "sports": "Physical activities involving skill, competition, and fitness.",
            "technology": "The application of scientific knowledge for practical purposes, particularly in industry.",
            "us_news": "News related to events, politics, and issues within the United States.",
            "world": "Global news, issues, and events occurring internationally."
        }

    @classmethod
    def get_description(cls, topic: "TopicEnum") -> str:
        """Fetches the description for a given topic using the class-level _descriptions dictionary."""
        descriptions = cls._get_descriptions()  # Safely fetch the descriptions
        # Convert spaces in topic.value to underscores to match the dictionary keys
        key = topic.value.replace(" ", "_")
        if key not in descriptions:
            raise ValueError(f"No description found for topic: {topic.value}")
        return descriptions[key]

class Schema(BaseModel):
    type: str
    properties: Dict[str, Any]
    required: list

class FunctionDeclarationWithExtra(BaseModel):
    name: str
    description: str
    parameters: Schema
    # Allow extra fields (like PROMPT)
    class Config:
        extra = Extra.allow
        
topic_list = "\n".join([f"{topic.value.capitalize()}: {TopicEnum.get_description(topic)}" for topic in TopicEnum])

# Define the function declaration for the topic classifier
topic_classifier_tool_handler_fn = FunctionDeclarationWithExtra(
    name="topic_classifier",
    description="Identify the article topic by analysing the contents of the article",
    parameters=Schema(
        type="OBJECT",
        properties={
            "article_text": {  # Define the expected 'article_text' input
                "type": "STRING",
                "description": "The content of the article to classify."
            },
            "topic": {
                "type": "STRING",
                "description": "Topic",
                "enum": [topic.value for topic in TopicEnum]
            },
        },
        required=["article_text", "topic"],
    ),
    PROMPT="""
    Identify the most relevant topic for the following article:

    {article_text}
    
    Review the enumerated topic categories and their descriptions categoried based upon both the topics and descriptions.
    If an areticle IS a poem, the topic is 'literature'.
    Only return one of the enumerated topics:
    
    {topic_list}
    
    """
)

# Function to classify article text based on topic classification
def classify_article_and_get_tags(article_text: str):
    # Dynamically set the prompt with the actual article text
    formatted_prompt = topic_classifier_tool_handler_fn.PROMPT.format(
        article_text=article_text,
        topic_list=topic_list
    )
    
    # Send the article text to the model for classification
    response = client.models.generate_content(
        model=MODEL,
        contents=formatted_prompt,
        config=GenerateContentConfig(
            response_mime_type="text/x.enum",
            response_schema={
                "type": "STRING",
                "enum": [topic.value for topic in TopicEnum],
            },
        ),
    )
    
    # The response contains the classification result
    classification_result = response.text.strip()  # Clean up the result
    
    return classification_result

# Now, assuming we have a dataframe or list of articles to process
article_data = [
    "You’ve heard of him, haven’t you? Phineas Gage. The railroad worker who survived an explosion...",
    "Another article text that talks about politics and economy...",
    # Add more articles as needed
]

## Generate the list
# print(topic_list)
# print

# Process each article and classify its topic
for idx, row in test.head(10).iterrows():
    article_text = row["text"]
    predicted_topic = classify_article_and_get_tags(article_text)
    actual_tags = row["tags"]  # Get the tags for the current row
    print(f"{article_text}\n")
    print("---------------------------------------------------------------------------------\n")
    print(f"Predicted Article Topic: {predicted_topic}\t\tDataset tags: {actual_tags}\n")
    print("---------------------------------------------------------------------------------\n\n")

Lloyd Austin, Secretary of Defense

While Gen. Lloyd Austin would be the first Black Defense Secretary, his terrible record should overshadow this feat, but we know that is going to be brought up in mainstream media. As Gen. Austin was tapped by Obama to begin the process of the withdrawal of American troops from Iraq in 2010, it was an unorganized effort leaving a depleted Iraqi government vulnerable with a military not trained or equipped to quell the rise of ISIS. The disaster did not stop there as Gen. Austin was then given the responsibility of overseeing a Syrian rebel program to combat ISIS that cost us $384 million, which ended in failure after the millions of dollars earmarked to build the training camps were rarely if ever used.

Shortly after all of these debacles, Gen. Austin sold out to defense contractors and wealthy investment funds becoming a board member at Raytheon and a partner at Pine Island Capital Partners, who both stand to profit immensely from a Biden administr

#### Sport and Score Extractor

input: article body  
output: identified sport, score summary

In [ ]:

# Import necessary modules from pydantic
from pydantic import BaseModel, Extra
from typing import Dict, Any
import pandas as pd  # Assuming you're using pandas for the test dataframe

# Define the Schema class
class Schema(BaseModel):
    type: str
    properties: Dict[str, Any]
    required: list

# Define the FunctionDeclarationWithExtra class
class FunctionDeclarationWithExtra(BaseModel):
    name: str
    description: str
    parameters: Schema
    # Allow extra fields (like PROMPT)
    class Config:
        extra = Extra.allow

# Define the function declaration for score extraction
score_extraction_func = FunctionDeclarationWithExtra(
    name="score_extraction",
    description="Extract the scores from the article if feasible",
    parameters=Schema(
        type="OBJECT",
        properties={
            "article_text": {  # Define the expected 'article_text' input
                "type": "STRING",
                "description": "The content of the article to extract scores."
            },
            "score_available": {
                "type": "BOOLEAN",
                "description": "Indicates if scores are available in the article."
            },
            "team1_score": {
                "type": "INTEGER",
                "description": "Score for team 1 if available."
            },
            "team2_score": {
                "type": "INTEGER",
                "description": "Score for team 2 if available."
            },
            "team1_name": {
                "type": "STRING",
                "description": "Name of team 1 if mentioned."
            },
            "team2_name": {
                "type": "STRING",
                "description": "Name of team 2 if mentioned."
            },
        },
        required=["article_text", "score_available"],
    ),
    PROMPT="""
    Given the following article, extract the score if feasible:

    {article_text}

    Provide the output in a structured JSON format:
    {{
        "score_available": true/false,
        "team1_name": "name_of_team1",
        "team1_score": score_of_team1,
        "team2_name": "name_of_team2",
        "team2_score": score_of_team2
    }}
    """
)

def extract_scores_from_article(article_text: str):
    # Dynamically set the prompt with the actual article text
    formatted_prompt = score_extraction_func.PROMPT.format(article_text=article_text)

    # Example client and content generation logic (ensure the client and model are correctly defined)
    response = client.models.generate_content(
        model=MODEL,
        contents=formatted_prompt,
        config=GenerateContentConfig(
            response_mime_type="application/json",
            response_schema={
                "type": "OBJECT",
                "properties": {
                    "score_available": {
                        "type": "BOOLEAN",
                    },
                    "team1_name": {
                        "type": "STRING",
                    },
                    "team1_score": {
                        "type": "INTEGER",
                    },
                    "team2_name": {
                        "type": "STRING",
                    },
                    "team2_score": {
                        "type": "INTEGER",
                    }
                },
                "required": ["score_available"],
            },
        ),
    )

    # The response contains the scores extracted from the article
    scores_extraction_result = json.loads(response.json())  # Parse the JSON response

    return scores_extraction_result

# Loop through the test dataframe and extract scores
for idx, row in test.head(2000).iterrows():  # Adjust the dataframe as necessary
    article_text = row["text"]
    extracted_scores = extract_scores_from_article(article_text)
    actual_tags = row["tags"]  # Get the tags for the current row

    if extracted_scores.get("score_available"):
        print(f"Extracted Scores: {extracted_scores}\t\tActual tags: {actual_tags}")

#### Sport Statistics Summarizer

input: article body  
output: statistics summary

In [ ]:

# Import necessary modules from pydantic
from pydantic import BaseModel, Extra
from typing import Dict, Any
import pandas as pd  # Assuming you're using pandas for the test dataframe
import json

# Define the Schema class
class Schema(BaseModel):
    type: str
    properties: Dict[str, Any]
    required: list

# Define the FunctionDeclarationWithExtra class
class FunctionDeclarationWithExtra(BaseModel):
    name: str
    description: str
    parameters: Schema
    # Allow extra fields (like PROMPT)
    class Config:
        extra = Extra.allow

# Define the function declaration for sport mention and person summary extraction
sport_person_extraction_func = FunctionDeclarationWithExtra(
    name="sport_person_extraction",
    description="Extract mentions of sports and summarize information related to individuals mentioned in the article if feasible",
    parameters=Schema(
        type="OBJECT",
        properties={
            "article_text": {  # Define the expected 'article_text' input
                "type": "STRING",
                "description": "The content of the article to extract sport mentions and person summaries."
            },
            "sport_mentioned": {
                "type": "BOOLEAN",
                "description": "Indicates if any sports are mentioned in the article."
            },
            "person_name": {
                "type": "STRING",
                "description": "Name of the person mentioned in relation to the sport."
            },
            "summary": {
                "type": "STRING",
                "description": "Summary of what is said about the person."
            },
        },
        required=["article_text", "sport_mentioned"],
    ),
    PROMPT="""
    Given the following article, identify if any sports are mentioned. If there are mentions of sports, find any names attached to the article and provide a summary about what the article is saying about that person.

    {article_text}

    Provide the output in a structured JSON format:
    {{
        "sport_mentioned": true/false,
        "person_name": "name_of_person_mentioned",
        "summary": "summary_about_person"
    }}
    """
)

def extract_sport_person_summary_from_article(article_text: str):
    # Dynamically set the prompt with the actual article text
    formatted_prompt = sport_person_extraction_func.PROMPT.format(article_text=article_text)

    # Example client and content generation logic (ensure the client and model are correctly defined)
    response = client.models.generate_content(
        model=MODEL,  # Make sure to replace with the actual model name you are using
        contents=formatted_prompt,
        config=GenerateContentConfig(
            response_mime_type="application/json",
            response_schema={
                "type": "OBJECT",
                "properties": {
                    "sport_mentioned": {
                        "type": "BOOLEAN",
                    },
                    "person_name": {
                        "type": "STRING",
                    },
                    "summary": {
                        "type": "STRING",
                    }
                },
                "required": ["sport_mentioned"],
            },
        ),
    )

    # Parse the JSON response
    response_data = json.loads(response.json())
    sport_person_extraction_result = response_data.get('parsed', {})

    return sport_person_extraction_result

# Loop through the test dataframe and extract sport mentions and person summaries
for idx, row in test.head(2000).iterrows():
    article_text = row["text"]
    extracted_info = extract_sport_person_summary_from_article(article_text)
    actual_tags = row["tags"]  # Get the tags for the current row

    if extracted_info.get("sport_mentioned") == True:
        print(f"Extracted Info: {extracted_info}\t\tActual tags: {actual_tags}")

### API Deployment (Input/Output)

Webapp endpoint

#### Support Pipeline

##### Dockerize LLM

##### Stand up Vertex Endpoint

## Reference Labs

- Gemini Function Calling
- Gemini Prompt Engineering
- AutoML for Text Classification - Vertex
- KFP Walkthrogh - Vertex Containerization - Training and Deployment Pipelines